# Übung - Metadaten gruppieren & auswerten 

Als nächstes wollen wir uns eine Liste von den Datenbereitstellern ansehen, sowie die Anzahl an Datensätzen je Bereitsteller. Damit können wir herausfinden, welcher Bereitstellende "am produktivsten" ist. 

In [1]:
%endpoint https://data.europa.eu/sparql
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?datasetTitle) AS ?uniqueDatasetTitleCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(CONTAINS(STR(?modified), "2022") || CONTAINS(STR(?modified), "2023") || CONTAINS(STR(?modified), "2024"))
}
GROUP BY ?contributorID

Endpoint set to: https://data.europa.eu/sparql

```{admonition} Erklärung des Codes
:class: tip, dropdown
Der COUNT Befehl in SPARQL wird verwendet, um die Anzahl der Ergebnisse zu zählen, die eine bestimmte Bedingung erfüllen. Der AS Befehl folgt dem COUNT und dient dazu, das Ergebnis der Zählung einer Variablen zuzuweisen, die dann im Ergebnis verwendet werden kann. DISTINCT gibt an, dass nur eindeutige (DISTINCT) Werte der Variable ?datasetTitle gezählt werden sollen. AS ?uniqueDatasetTitleCount  weist das Ergebnis der Zählung der Variable ?uniqueDatasetTitleCount zu. Diese Variable kann dann im Ergebnis verwendet werden, um die Anzahl der eindeutigen datasetTitle für jeden contributorID anzuzeigen. Die Bedingung (LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") wird verwendet, um sicherzustellen, dass nur Titel (?datasetTitle) ausgewählt werden, die entweder keine Sprachinformation haben oder explizit als Deutsch ("de") gekennzeichnet sind. Die Bedingung für das Aufnehmen von Ergebnisse mit keiner Sprachinformation ist auch wichtig, da nicht alle Datenbereitsteller Information über die Sprache des Datensatzes einschließen. Die Bedingung, dass nur Datensätze als jene auf Deutsch aufgelistet werden sollen, ist vielleicht in diesem Fall überflüssig, denn das Wort "Baumkataster" sollte auch erwähnt werden, also ein deutsches Wort. Trotzdem wollen wir diese Funktion hierbei hervorheben, weil wir später davon Gebrauch machen.
```

Es erweist sich, dass die Datenbereitsteller Open Data Bayern und Open NRW, also die zwei deutsche Bundesländer, die größte Anzahl an Datensätzen leisten, mit jeweils 5. Das Ergebnis dieser Aufsummierung klingt plausibel, da Bayern und NRW die zwei größten Bundesländer in Deutschland sind.

In [3]:
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX dcatde: <http://dcat-ap.de/def/dcatde/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX pg: <http://dcat-ap.de/def/politicalGeocoding/>

SELECT ?contributorID (COUNT(DISTINCT ?format) AS ?formatCount)
WHERE {
  ?datasetURI a dcat:Dataset;
              dct:title ?datasetTitle;
              dcatde:contributorID ?contributorID;
              dct:modified ?modified.
  OPTIONAL { ?datasetURI dcat:catalog ?catalog. }
  FILTER((LANG(?datasetTitle) = "" || LANG(?datasetTitle) = "de") && CONTAINS(LCASE(?datasetTitle), "baumkataster"))
  FILTER(CONTAINS(STR(?modified), "2022") || CONTAINS(STR(?modified), "2023") || CONTAINS(STR(?modified), "2024"))

  ?datasetURI dcat:distribution ?distribution.
  ?distribution dct:format ?format.
}
GROUP BY ?contributorID

contributorID,formatCount
http://dcat-ap.de/def/contributors/gdiDE,2
https://offenedaten-konstanz.de/,2
http://dcat-ap.de/def/contributors/schleswigHolstein,4
http://dcat-ap.de/def/contributors/openNRW,26
http://dcat-ap.de/def/contributors/openDataBayern,21
http://dcat-ap.de/def/contributors/stadtFrankfurtAmMain,1
http://dcat-ap.de/def/contributors/mobilithek,1
https://gdi-sh.de,3
http://dcat-ap.de/def/contributors/freistaatSachsen,1
http://dcat-ap.de/def/contributors/stadtErlangen,1


Anbei auch die Ergebnisse der aufsummierten Datenformate je Datenbereitsteller. Nicht überraschend festzustellen ist, dass NRW und Bayern die breitesten Auswahl an Datenformaten leisten. Damit wird nicht impliziert, dass alle Datensätze in jeweils jedem Format vorhanden sind. Jedoch dient das als eine Maß für Flexibilität. 